In this kernel I will share how I managed to build stacking model wich consists on linear regression with regularization and xgboost. I will use alredy cleaned and prepared train data 'all_data_final.pkl' from my another kernel. https://www.kaggle.com/emaksone/eda-with-feature-engineering this is how obtained it.

Pipeline:

* Define validation strategy and leaderboard probing
* Hyperparameter tuning
* Stacking


In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
from sklearn.externals import joblib

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from xgboost import XGBRegressor

import gc
from itertools import product
import time

In [2]:
all_data = pd.read_pickle('../input/extended-train-data/all_data_final.pkl')
all_data.head()

,shop_id,item_id,date_block_num,target,is_december,item_category_id,type_code,subtype_code,city_code,date_block_num_enc_lag_1,date_block_num_enc_lag_2,date_block_num_enc_lag_3,date_block_num_enc_lag_6,date_block_num_enc_lag_12,date_block_num_item_enc_lag_1,date_block_num_shop_enc_lag_1,date_block_num_cat_enc_lag_1,date_block_num_shop_cat_enc_lag_1,date_block_num_shop_type_enc_lag_1,date_block_num_shop_subtype_enc_lag_1,date_block_num_city_enc_lag_1,date_block_num_item_city_enc_lag_1,date_block_num_type_enc_lag_1,date_block_num_subtype_enc_lag_1,date_block_num_item_enc_lag_2,date_block_num_shop_enc_lag_2,date_block_num_cat_enc_lag_2,date_block_num_shop_cat_enc_lag_2,date_block_num_shop_type_enc_lag_2,date_block_num_shop_subtype_enc_lag_2,date_block_num_city_enc_lag_2,date_block_num_item_city_enc_lag_2,date_block_num_type_enc_lag_2,date_block_num_subtype_enc_lag_2,date_block_num_item_enc_lag_3,date_block_num_shop_enc_lag_3,date_block_num_cat_enc_lag_3,date_block_num_shop_cat_enc_lag_3,date_block_num_shop_type_enc_lag_3,date_block_num_shop_subtype_enc_lag_3,date_block_num_city_enc_lag_3,date_block_num_item_city_enc_lag_3,date_block_num_type_enc_lag_3,date_block_num_subtype_enc_lag_3,date_block_num_item_enc_lag_6,date_block_num_shop_enc_lag_6,date_block_num_cat_enc_lag_6,date_block_num_shop_cat_enc_lag_6,date_block_num_shop_type_enc_lag_6,date_block_num_shop_subtype_enc_lag_6,date_block_num_city_enc_lag_6,date_block_num_item_city_enc_lag_6,date_block_num_type_enc_lag_6,date_block_num_subtype_enc_lag_6,date_block_num_item_enc_lag_12,date_block_num_shop_enc_lag_12,date_block_num_cat_enc_lag_12,date_block_num_shop_cat_enc_lag_12,date_block_num_shop_type_enc_lag_12,date_block_num_shop_subtype_enc_lag_12,date_block_num_city_enc_lag_12,date_block_num_item_city_enc_lag_12,date_block_num_type_enc_lag_12,date_block_num_subtype_enc_lag_12,delta_shop_price_lag,delta_item_price_lag,month,days,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
4488710,54,10297,12,4.0,0,37,11,1,26,0.411377,0.303223,0.000000,0.000000,0.0,0.913086,0.989258,0.232788,0.585938,0.727539,0.579590,0.989258,3.0,0.270020,0.234009,0.044434,0.796387,0.183105,0.589844,0.720703,0.588867,0.796387,0.0,0.221436,0.183472,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132324,0.037933,0,31,1,1,2,2
4488711,54,10296,12,3.0,0,38,11,1,26,0.411377,0.000000,0.000000,0.000000,0.0,0.521973,0.989258,0.243652,0.551758,0.727539,0.579590,0.989258,0.0,0.270020,0.234009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132324,0.085205,0,31,-1,1,1,1
4488712,54,10298,12,14.0,0,40,11,4,26,0.411377,0.303223,0.290039,0.000000,0.0,7.218750,0.989258,0.291504,0.822754,0.727539,0.822754,0.989258,20.0,0.270020,0.291504,15.023438,0.796387,0.244873,0.812988,0.720703,0.812988,0.796387,20.0,0.221436,0.244873,3.130859,0.6875,0.236206,0.706543,0.639160,0.706543,0.6875,7.0,0.209961,0.236206,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132324,0.439209,0,31,1,1,3,3
4488713,54,10300,12,3.0,0,37,11,1,26,0.411377,0.303223,0.290039,0.000000,0.0,1.173828,0.989258,0.232788,0.585938,0.727539,0.579590,0.989258,1.0,0.270020,0.234009,7.265625,0.796387,0.183105,0.589844,0.720703,0.588867,0.796387,20.0,0.221436,0.183472,1.152344,0.6875,0.170776,0.535645,0.639160,0.539062,0.6875,0.0,0.209961,0.169189,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132324,0.160645,0,31,1,1,3,3
4488714,54,10284,12,1.0,0,57,13,7,26,0.411377,0.303223,0.290039,0.276855,0.0,0.086975,0.

**<font size=4>Validation strategy</font>**

Our object is to predict sales for next month (i.e. November 2015, or 34 in terms of date_block_num). And we can't just randomly split our data on tran/validation sets for parameters tuning because we need validate parameters on data in month that we didn't use in train set like in test. So,I use rows with date_block_num less than 33 as train set and rows with date_block_num equal 33 I use as validation set.

To successfully predict sales on test data you also need to have same train and test data. In other words you train and test data should have the same distribution. But stop, distribution in train data we know but how do we know distribution in test data? Such information we can get by **leaderboard probing**. For example we can easily find out mean of sales in public leader board by simlpy submiting two files with all 0's prediction and 0.5's predictions and do some math.


And the mean of sales equals 0.28394. So mean of train/validation should be close to this number in order to get good results. Let's check mean of the target in our whole train set

In [34]:
print('mean for whole train set: {0}'.format(np.mean(all_data.loc[all_data['date_block_num']<34, 'target'].astype(np.float32))))
print('mean for validation train set: {0}'.format(np.mean(all_data.loc[all_data['date_block_num']<33, 'target'].astype(np.float32))))
print('mean for validation test set: {0}'.format(np.mean(all_data.loc[all_data['date_block_num']==33, 'target'].astype(np.float32))))

mean for whole train set: 0.28772589564323425
mean for validation train set: 0.28884848952293396
mean for validation test set: 0.2585652470588684


As you can see every numbers are close to mean of test sales. 

One thing to notice before start building a model. Let's check if we have new items in test set

In [4]:
test_items = all_data.loc[all_data['date_block_num']==34,'item_id'].unique()
train_items = all_data.loc[all_data['date_block_num']<34,'item_id'].unique()
items_in_test_and_not_in_train = set(test_items).difference(set(train_items))
print('Items in test and not in train: {0}'.format(len(items_in_test_and_not_in_train)))
items_in_train_and_not_in_test = set(train_items).difference(set(test_items))
print('Items in train and not in test: {0}'.format(len(items_in_train_and_not_in_test)))

test_shops = all_data.loc[all_data['date_block_num']==34,'shop_id'].unique()
print('Number of unique shops: {0}'.format(len(test_shops)))

Items in test and not in train: 378
Items in train and not in test: 12332
Number of unique shops: 42


So, we have items in the test set that do not appear in train. So our model will struggle when see unknown items when predict. To eliminate this issue I will add all missing shop/item pair to every month with 0's.
But after this we decrese our target mean that will make our train/test data distributions different. To avoid this I remove rows with target equals 0 and item that does not appear in test. At the end it doesn't change mean of targets and train data set will contain all items that exist in test.

In [5]:
missing_shop_item_count = 15876 # 372*42 all missing items per month
index_cols = ['shop_id', 'item_id', 'date_block_num']

grid = [] 
for block_num in all_data.loc[all_data['date_block_num']<34, 'date_block_num'].unique():
    print(block_num)
  
    zero_target_df = all_data[(all_data['date_block_num'] == block_num) & (all_data['target']==0) & 
                              (all_data['item_id'].isin(items_in_train_and_not_in_test))]

    idx_to_delete = zero_target_df.sample(missing_shop_item_count, random_state=block_num).index
    all_data.drop(idx_to_delete, inplace=True)
    temp = np.array(list(product(*[test_shops, items_in_test_and_not_in_train, [block_num]])),dtype='int32')
    grid.append(temp)
    
    del zero_target_df
    del idx_to_delete
    del temp
    gc.collect()

grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [6]:
grid['shop_id'] = grid['shop_id'].astype(np.int16)
grid['item_id'] = grid['item_id'].astype(np.int32)
grid['date_block_num'] = grid['date_block_num'].astype(np.int8)

In [7]:
all_data = pd.concat([all_data, grid], ignore_index=True, sort=False, keys=index_cols)
all_data[['item_shop_last_sale', 'item_last_sale']].fillna(-1, inplace=True) #-1 is default value in this columns
all_data.fillna(0, inplace=True)

del grid
del test_items
del test_shops
del train_items
del items_in_test_and_not_in_train
del items_in_train_and_not_in_test
gc.collect()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


7

In [8]:
all_data['is_december'] = all_data['is_december'].astype(np.int8)
all_data['item_category_id'] = all_data['item_category_id'].astype(np.int8)
all_data['type_code'] = all_data['type_code'].astype(np.int8)
all_data['subtype_code'] = all_data['subtype_code'].astype(np.int8)
all_data['city_code'] = all_data['city_code'].astype(np.int16)

all_data['month'] = all_data['month'].astype(np.int8)
all_data['days'] = all_data['days'].astype(np.int8)
all_data['item_shop_last_sale'] = all_data['item_shop_last_sale'].astype(np.int8)
all_data['item_last_sale'] = all_data['item_last_sale'].astype(np.int8)
all_data['item_shop_first_sale'] = all_data['item_shop_first_sale'].astype(np.int8)
all_data['item_first_sale'] = all_data['item_first_sale'].astype(np.int8)

Check our means

In [9]:
print('mean for whole train set: {0}'.format(np.mean(all_data.loc[all_data['date_block_num']<34, 'target'].astype(np.float32))))
print('mean for validation train set: {0}'.format(np.mean(all_data.loc[all_data['date_block_num']<33, 'target'].astype(np.float32))))
print('mean for validation test set: {0}'.format(np.mean(all_data.loc[all_data['date_block_num']==33, 'target'].astype(np.float32))))

mean for whole train set: 0.28772589564323425
mean for validation train set: 0.28884848952293396
mean for validation test set: 0.2585652470588684


Good, As it was before.

In [10]:
# put added rows in right position
all_test_data = all_data[all_data['date_block_num'] == 34]
all_data = all_data[all_data['date_block_num'] < 34]
all_data.sort_values(['date_block_num'], inplace=True)
all_data = pd.concat([all_data, all_test_data], ignore_index=True, sort=False, keys=index_cols)

del all_test_data
gc.collect()

0

Now we ready to define train/test split. And after we tune parameters we will use whole data to train a model on this parameters and can use it to predict sales on test data.

In [11]:
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is {0}'.format(last_block))

X_train = all_data.loc[dates <  last_block]
X_test =  all_data.loc[dates == last_block]

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

X_valid_train = all_data.loc[dates <  last_block-1]
X_valid_test =  all_data.loc[dates == last_block-1]

y_valid_train = all_data.loc[dates <  last_block-1, 'target'].values
y_valid_test =  all_data.loc[dates == last_block-1, 'target'].values

all_data.to_pickle('all_data.pkl') # will use it later. Now free RAM

del dates
del all_data
gc.collect()

Test `date_block_num` is 34


0

Delete columns that we can't use in test

In [12]:
columns_to_delete = ['date_block_num', 'target']
X_valid_train = X_valid_train.drop(columns_to_delete, axis=1)
X_valid_test = X_valid_test.drop(columns_to_delete, axis=1)

X_train = X_train.drop(columns_to_delete, axis=1)
X_test = X_test.drop(columns_to_delete, axis=1)

**<font size=4>Hyperparameters tuning<font>**

I create my own custom grid search method because I use XGBoost with GPU and I didn't find a way to make them work together. If you know how to do it please drop a comment.

In [35]:
#validation
def validate(estimator, X_train_, y_train_, X_val_, y_val_, grid_params):
    keys = grid_params.keys()
    vals = grid_params.values()
    parameters = []
    rmses = []
    rmses_train = []
    return_obj ={}
    prods = product(*vals)
   
    for idx, instance in enumerate(prods):
        print('-'*50)
        print('model {0}:'.format(idx))
        model_params = dict(zip(keys, instance))
        parameters.append(model_params)
        
        print(model_params)
        model = estimator(**model_params)
        model.fit(X_train_, y_train_)
            
        pred_test = model.predict(X_val_)       
        mse = mean_squared_error(y_val_, pred_test)
        rmse = np.sqrt(mse)
        print('RMSE: {0}'.format(rmse))
        rmses = rmses + [rmse]
        
        best_rmse_so_far = np.min(rmses)
        print('Best rmse so far: {0}'.format(best_rmse_so_far))
        best_model_params_so_far = parameters[np.argmin(rmses)]
        print('Best model params so far: {0}'.format(best_model_params_so_far))
        
        del best_rmse_so_far
        del best_model_params_so_far
        del pred_test
        del model
        gc.collect()
    
    rmses = np.array(rmses)
    best_rmse = np.min(rmses)
    print('Best rmse: {0}'.format(best_rmse))
    best_model_params = parameters[np.argmin(rmses)]
    print('Best model params: {0}'.format(best_model_params))

    return_obj['rmses'] = rmses
    return_obj['best_rmse'] = best_rmse
    return_obj['best_model_params'] = best_model_params
      
    return return_obj

Let's find out optimal paramter for Ridge regression

In [14]:
alphas = [10, 100, 2000, 5000]
grid_params = {'alpha':alphas}
val_res = validate(Ridge, X_valid_train, y_valid_train, 
                   X_valid_test, y_valid_test, grid_params)

--------------------------------------------------
model 0:
{'alpha': 10}


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


RMSE: 0.9780671363589905
Best rmse so far: 0.9780671363589905
Best model params so far: {'alpha': 10}
--------------------------------------------------
model 1:
{'alpha': 100}


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


RMSE: 0.9780657035917624
Best rmse so far: 0.9780657035917624
Best model params so far: {'alpha': 100}
--------------------------------------------------
model 2:
{'alpha': 2000}


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


RMSE: 0.9780530491962817
Best rmse so far: 0.9780530491962817
Best model params so far: {'alpha': 2000}
--------------------------------------------------
model 3:
{'alpha': 5000}


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


RMSE: 0.9780588043518567
Best rmse so far: 0.9780530491962817
Best model params so far: {'alpha': 2000}
Best rmse: 0.9780530491962817
Best model params: {'alpha': 2000}


So 2000 is an optimal parameter for our model. We can use it to train model with all train data.

In [15]:
# best_alpha=2000
# ridge_model = Ridge(best_alpha)
# ridge_model.fit(X_train, y_train)
# predictions = ridge_model.predict(X_test)

In the same way we can do for XGBoost. But this model has much more parameters to config. https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/ this article helps to define strategy to find them starting with some parameters that make highest impact on the model and ending with ones that give lowest.
So I came up with this parameters. Note that tree_method, predictor, gpu_id was fixed parameters from the begining to enable gpu. It works extremely slow without it.

Note: Don't forget to enable gpu in kernel settings.

In [16]:
best_params = {'learning_rate': 0.16, 'n_estimators': 500, 
               'max_depth': 6, 'min_child_weight': 7,
               'subsample': 0.9, 'colsample_bytree': 0.7, 'nthread': -1, 
               'scale_pos_weight': 1, 'random_state': 42, 
               
               #next parameters are used to enable gpu for fasting fitting
               'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor', 'gpu_id': 0}

In [17]:
ts = time.time()
model = XGBRegressor(**best_params)
model.fit(X_valid_train.values, 
                y_valid_train,
                eval_metric="rmse", 
                eval_set=[(X_valid_test.values, y_valid_test)], 
                verbose=True, 
                early_stopping_rounds = 50)


time.time() - ts

[0]	validation_0-rmse:1.10977
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:1.06714
[2]	validation_0-rmse:1.03737
[3]	validation_0-rmse:1.01324
[4]	validation_0-rmse:0.995809
[5]	validation_0-rmse:0.981793
[6]	validation_0-rmse:0.96942
[7]	validation_0-rmse:0.962672
[8]	validation_0-rmse:0.957375
[9]	validation_0-rmse:0.952107
[10]	validation_0-rmse:0.948784
[11]	validation_0-rmse:0.946856
[12]	validation_0-rmse:0.944467
[13]	validation_0-rmse:0.943044
[14]	validation_0-rmse:0.941548
[15]	validation_0-rmse:0.940759
[16]	validation_0-rmse:0.940641
[17]	validation_0-rmse:0.939734
[18]	validation_0-rmse:0.939436
[19]	validation_0-rmse:0.937174
[20]	validation_0-rmse:0.934588
[21]	validation_0-rmse:0.932408
[22]	validation_0-rmse:0.931958
[23]	validation_0-rmse:0.931964
[24]	validation_0-rmse:0.931886
[25]	validation_0-rmse:0.931485
[26]	validation_0-rmse:0.932094
[27]	validation_0-rmse:0.932031
[28]	validation_0-rmse:0.931802
[29]	validation_0-rmse

[251]	validation_0-rmse:0.907082
[252]	validation_0-rmse:0.907106
[253]	validation_0-rmse:0.907086
[254]	validation_0-rmse:0.906989
[255]	validation_0-rmse:0.907141
[256]	validation_0-rmse:0.907
[257]	validation_0-rmse:0.906858
[258]	validation_0-rmse:0.906945
[259]	validation_0-rmse:0.90663
[260]	validation_0-rmse:0.906662
[261]	validation_0-rmse:0.9067
[262]	validation_0-rmse:0.906468
[263]	validation_0-rmse:0.906443
[264]	validation_0-rmse:0.906902
[265]	validation_0-rmse:0.906893
[266]	validation_0-rmse:0.906803
[267]	validation_0-rmse:0.906769
[268]	validation_0-rmse:0.906728
[269]	validation_0-rmse:0.906697
[270]	validation_0-rmse:0.906853
[271]	validation_0-rmse:0.906881
[272]	validation_0-rmse:0.906881
[273]	validation_0-rmse:0.907517
[274]	validation_0-rmse:0.907605
[275]	validation_0-rmse:0.90752
[276]	validation_0-rmse:0.907485
[277]	validation_0-rmse:0.907503
[278]	validation_0-rmse:0.908068
[279]	validation_0-rmse:0.908062
[280]	validation_0-rmse:0.908014
[281]	validation_

52.800353050231934

Look how better xgboost then simple regression model.

In [18]:
joblib.dump(model, 'xgboost_model.pkl')
del model
del X_train
del X_test
del y_train
del y_test
del X_valid_train
del X_valid_test
del y_valid_train
del y_valid_test

gc.collect()

68

**<font size=4>Stacking<font>**

So now after we find optimal parameters for two models we can use them for stacking. I will do it in the following way: 

* take data that date_block_num < 27 as train
* train a models(Ridge and XGBoost) and predict for date_block_num == 27
* put this prediction in two columns (xgb_prediction and ridge_prediction)
* do it for 28, 29, 30, 31, 32, 33, 34 monthes
* concat all this prediction.
* fit data from 28 to 33 to Linear regression
* use this simple model to predict data with month 34

In [19]:
all_data = pd.read_pickle('all_data.pkl')

In [20]:
def compute_prediction_for_specifi_month(monthes_before, df, estimator, params, prefix):
    predictions = pd.DataFrame(columns=['date_block_num', 'pred_'+prefix])
    for before in monthes_before:
        last_valid_month = np.max(df['date_block_num'])
        print('train: 12 to {0}'.format(last_valid_month-before-1))
        print('test: {0}'.format(last_valid_month-before))
        
        cur_train = df[df['date_block_num'] < last_valid_month-before]
        cur_test = df[df['date_block_num'] == last_valid_month-before]
        
        cur_y_train = cur_train['target']
        cur_train.drop('target', axis=1, inplace=True)
        cur_test.drop('target', axis=1, inplace=True)
        
        model = estimator(**params)
        model.fit(cur_train.values, cur_y_train.values)
        pred = model.predict(cur_test.values)
        cur_df = pd.DataFrame(columns=['date_block_num', 'pred_'+prefix])
        
        cur_df['pred_'+prefix] = pred
        cur_df['date_block_num'] = (last_valid_month-before)
       
        predictions = pd.concat([predictions, cur_df])
        del model
        del pred
        del cur_test
        del cur_df
        del cur_y_train
        gc.collect()
        
    return predictions

In [21]:
best_param = {
    'learning_rate' :0.16,
    'n_estimators':500,
    'max_depth':6,
    'min_child_weight':7,
    'subsample':0.9,
    'colsample_bytree':0.7,
    'nthread':-1,
    'scale_pos_weight':1,
     #next parameters are used to enable gpu for fasting fitting
    'random_state':42,
    'nthread': -1,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'gpu_id': 0,
}

ts= time.time()
monthes_before = [6, 5, 4, 3, 2, 1, 0]

stacks_xgb = compute_prediction_for_specifi_month(monthes_before, all_data, XGBRegressor, best_params,'xgb')
time.time()-ts

train: 12 to 27
test: 28


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


train: 12 to 28
test: 29
train: 12 to 29
test: 30
train: 12 to 30
test: 31
train: 12 to 31
test: 32
train: 12 to 32
test: 33
train: 12 to 33
test: 34


461.87108540534973

Clip predictions

In [22]:
stacks_xgb['pred_xgb'] = stacks_xgb['pred_xgb'].clip(0, 20)

And do it for second model

In [23]:
ts = time.time()
monthes_before = [6, 5, 4, 3, 2, 1, 0]
best_param ={'alpha': 2000}
stacks_lin_reg = compute_prediction_for_specifi_month(monthes_before, all_data, Ridge, best_param,'lin')
time.time() - ts

train: 12 to 27
test: 28


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


train: 12 to 28
test: 29


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


train: 12 to 29
test: 30


...

train: 12 to 32
test: 33


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


train: 12 to 33
test: 34


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


121.0373706817627

Clip predictions again

In [26]:
stacks_lin_reg['pred_lin'] = stacks_lin_reg['pred_lin'].clip(0, 20)

And construct train/validation/test sets

In [27]:
X_valid_train = pd.concat([stacks_xgb[stacks_xgb['date_block_num'] < 33], 
                           stacks_lin_reg[stacks_lin_reg['date_block_num'] < 33]], axis=1)
X_valid_test = pd.concat([stacks_xgb[stacks_xgb['date_block_num'] == 33], 
                           stacks_lin_reg[stacks_lin_reg['date_block_num'] == 33]], axis=1)
y_valid_train = all_data.loc[(all_data['date_block_num']<33) & (all_data['date_block_num']>27), 'target']
y_valid_test = all_data.loc[all_data['date_block_num']==33, 'target']

In [28]:
X_train = pd.concat([stacks_xgb[stacks_xgb['date_block_num'] < 34], 
                           stacks_lin_reg[stacks_lin_reg['date_block_num'] < 34]], axis=1)
X_test = pd.concat([stacks_xgb[stacks_xgb['date_block_num'] == 34], 
                           stacks_lin_reg[stacks_lin_reg['date_block_num'] == 34]], axis=1)
y_train = all_data.loc[(all_data['date_block_num']<34) & (all_data['date_block_num']>27), 'target']

In [29]:
X_valid_train.drop('date_block_num', axis=1, inplace=True)
X_valid_test.drop('date_block_num', axis=1, inplace=True)
X_train.drop('date_block_num', axis=1, inplace=True)
X_test.drop('date_block_num', axis=1, inplace=True)

And use simple linear regression.

In [30]:
model = LinearRegression()
model.fit(X_valid_train, y_valid_train)
pred = model.predict(X_valid_test).clip(0,20)

rmse = np.sqrt(mean_squared_error(y_valid_test, pred))
print('RMSE on valid set: {0}'.format(rmse))

RMSE on valid set: 0.900349656662716


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


As you see the rmse is lower then single xgboost model's rmse.

Let's train a stacking model on a whole train data.

In [31]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test).clip(0,20)

/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


In [32]:
submit = pd.DataFrame({'ID':range(len(pred)), 'item_cnt_month': pred})
submit.to_csv('submit.csv', index=False)

**<font size=4>Summary</font>**

In this kernel we learned how to make proper cross validation strategy, tune parameters for single models and finnaly use one of the ensambling methods called stacking to make better predictions.